# Cira playground

In [1]:
import cira

In [2]:
cira.auth.KEY_FILE = "../alpc_key.json"

assert cira.auth.check_keys(), "the set keys dose not work"

In [3]:
portfolio = cira.Portfolio()
exchange = cira.Exchange()

TypeError: 'bool' object is not callable

In [ ]:
cry = cira.Cryptocurrency("BTC/USD")    
print(cry,cry.current_price())

spy = cira.Stock("SPY")
print(spy, spy.current_price())

BTC/USD 0.0
SPY 456.45


In [ ]:
print(portfolio.cash())
pos = cira.Position("GOOGL")
print(pos, pos.quantity())

17597.6
GOOGL 1360


In [ ]:
#exchange.get_all_stocks()[:5] # is cached but will do request first time 

In [ ]:
from datetime import datetime

start = datetime(2022, 7, 1)
end = datetime(2022, 7, 10)
spy.historical_data_df(start, end).head()

open    high      low   close      volume  \
symbol timestamp                                                                
SPY    2022-07-01 04:00:00+00:00  376.56  381.70  373.800  381.24  74084447.0   
       2022-07-05 04:00:00+00:00  375.88  381.98  372.900  381.96  79886359.0   
       2022-07-06 04:00:00+00:00  382.11  385.87  379.600  383.25  69842806.0   
       2022-07-07 04:00:00+00:00  385.12  389.83  383.267  388.99  64172156.0   
       2022-07-08 04:00:00+00:00  387.27  390.64  385.660  388.67  71954884.0   

                                  trade_count        vwap  
symbol timestamp                                           
SPY    2022-07-01 04:00:00+00:00     600889.0  378.221933  
       2022-07-05 04:00:00+00:00     645355.0  377.447571  
       2022-07-06 04:00:00+00:00     614418.0  382.508246  
       2022-07-07 04:00:00+00:00     467708.0  387.621834  
       2022-07-08 04:00:00+00:00     509773.0  388.651531

In [ ]:
exchange.is_open()

False

In [ ]:
portfolio.buying_power()

AttributeError: 'Portfolio' object has no attribute 'buying_power'